# Домашнее задание № 3. Исправление опечаток

In [1]:
import re
from string import punctuation
punctuation += "«»—…“”"
from collections import Counter
import textdistance
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_distances
from tqdm.notebook import tqdm

## 1. Доп. ранжирование по вероятности (3 балла)

Дополните get_closest_hybrid_match в семинаре так, чтобы из кандадатов с одинаковым расстоянием редактирования выбиралось наиболее вероятное.

Исходные функции

In [5]:
def get_closest_match_with_metric(text, lookup, topn=20, metric=textdistance.levenshtein):
    # Counter можно использовать и с не целыми числами
    similarities = Counter()
    
    for word in lookup:
        similarities[word] = metric.normalized_similarity(text, word) 
    
    return similarities.most_common(topn)

def get_closest_hybrid_match(text, X, vec, topn=3, metric=textdistance.damerau_levenshtein):
    candidates = get_closest_match_vec(text, X, vec, topn*4)
    lookup = [cand[0] for cand in candidates]
    closest = get_closest_match_with_metric(text, lookup, topn, metric=metric)
    
    return closest

def get_closest_match_vec(text, X, vec, topn=20):
    v = vec.transform([text])
    
    # вся эффективноть берется из того, что мы сразу считаем близость 
    # 1 вектора ко всей матрице (словам в словаре)
    # считать по отдельности циклом было бы дольше
    # вместо одного вектора может даже целая матрица
    # тогда считаться в итоге будет ещё быстрее
    
    similarities = cosine_distances(v, X)[0]
    topn = similarities.argsort()[:topn] 
    
    return [(id2word[top], similarities[top]) for top in topn]

def P(word, N=N):
    return vocab[word] / N

def predict_mistaken(word, vocab):
    return 0 if word in vocab else 1

Создать словарь и дистрибутивную матрицу

In [3]:
with open('wiki_data.txt', encoding='utf-8') as f:
    corpus = f.read()

vocab = Counter(re.findall('\w+', corpus.lower()))
N = sum(vocab.values())

word2id = list(vocab.keys())
id2word = {i:word for i, word in enumerate(vocab)}

vec = CountVectorizer(analyzer='char', max_features=10000, ngram_range=(1,3))
X = vec.fit_transform(vocab)

In [4]:
vocab.most_common(10)

[('в', 267296),
 ('и', 147115),
 ('на', 81926),
 ('с', 61681),
 ('года', 43894),
 ('по', 37235),
 ('году', 32197),
 ('из', 29150),
 ('был', 23293),
 ('не', 23228)]

Обновлённая функция. Надо выбрать более вероятный (частотный) вариант в случаях типа следующего

In [6]:
get_closest_hybrid_match('пртер', X, vec)

[('партер', 0.8333333333333334),
 ('портер', 0.8333333333333334),
 ('партере', 0.7142857142857143)]

In [7]:
print('партер', P('партер'))
print('портер', P('портер')) #частотнее по корпусу

партер 1.3578314578124677e-06
портер 3.1036147607142114e-06


In [8]:
def NEW_get_closest_hybrid_match(text, X, vec, topn=3, metric=textdistance.damerau_levenshtein):
    candidates = get_closest_match_vec(text, X, vec, topn*4)
    lookup = [cand[0] for cand in candidates]
    closest = get_closest_match_with_metric(text, lookup, topn, metric=metric)
    
    candidates_id = [0]
    the_closest = closest[0][1]
    for i in range(1, len(closest)):
        if closest[i][1] < the_closest:
            break
        elif closest[i][1] == the_closest:
            candidates_id.append(i)
        else:
            return 'Wrong input from get_closest_match_with_metric'
    
    most_probable = closest[candidates_id[0]]
    for i in range(1, len(candidates_id)):
        if P(closest[candidates_id[i]][0]) > P(most_probable):
            most_probable = closest[candidates_id[i]]
    
    return most_probable

In [9]:
NEW_get_closest_hybrid_match('пртер', X, vec)

('портер', 0.8333333333333334)

Пример, когда первый кандидат самый близкий и не нужно переходить к частотности

In [10]:
get_closest_hybrid_match('сcолнце', X, vec)

[('солнце', 0.8571428571428572),
 ('солнцем', 0.7142857142857143),
 ('солнцев', 0.7142857142857143)]

In [11]:
NEW_get_closest_hybrid_match('сcолнце', X, vec)

('солнце', 0.8571428571428572)

## 2.  Symspell (7 баллов)

Реализуйте алгоритм Symspell. Он похож на алгоритм Норвига, но проще и быстрее. Он основан только на одной операции - удалении символа. Описание алгоритма по шагам:

1) Составляется словарь правильных слов

2) На основе словаря правильных слов составляется словарь удалений - для каждого правильного слова создаются все варианты удалений и создается словарь, где ключ - слово с удалением, а значение - правильное слово (!)

3) Для выбора исправления для слова с опечаткой генерируются все варианты удаления, из них выбираются те, что есть в словаре удалений, построенного на шаге 2. Слово с опечаткой заменяется на правильное слово, соответствующее варианту удаления

4) Если в словаре удалений есть несколько вариантов, то выбирается удаление, которому соответствует наиболее вероятное правильное слово  


Оцените качество полученного алгоритма теми же тремя метриками.

Словарь правильных слов (условно: по корпусу)

In [12]:
correct_vocab = set(i for i in vocab)

for i in list(correct_vocab)[:10]:
    print(i)

arne
дьявола
некадровым
остриё
артинские
кардиналом
телефоном
авачинскую
писаренко
лягушачьи


Словарь удалений (только 1 удаление, длина слова не учитывается)

In [14]:
deletions_vocab = dict()
for w in correct_vocab:
    for char in w:
        deletions_vocab[w.replace(char, '')] = w

for i in list(deletions_vocab.items())[:10]:
    print(i)

('rne', 'rune')
('ane', 'jane')
('are', 'arge')
('arn', 'arin')
('ьявола', 'дьявола')
('дявола', 'дьявола')
('дьвола', 'дьявола')
('дьяола', 'дьявола')
('дьявла', 'дьявола')
('дьявоа', 'дьявола')


Функция

In [15]:
def get_deletions(word):
    deletions = []
    for char in word:
        deletions.append(word.replace(char, ''))
    return deletions

def get_most_probable_word(candidates):    
    most_probable = candidates[0]
    for i in range(1, len(candidates)):
        if P(candidates[i]) > P(most_probable):
            most_probable = candidates[i]
    return most_probable

def get_corrected_word(word, correct_vocab, deletions_vocab):
    if predict_mistaken(word, correct_vocab) == False:
        return word
    else:
        candidates = []
        for i in get_deletions(word):
            if deletions_vocab.get(i):
                candidates.append(deletions_vocab.get(i))
        print(candidates)
        
        if len(candidates) == 0:
            return word
        elif len(candidates) == 1:
            return deletions_vocab.get(candidates[0])
        else:
            return get_most_probable_word(candidates)

Я постарался в точности реализовать алгоритм, описанный в задании. Но у меня сложилось впечатление, что там не совсем корректная  формулировка: "<...> 3) Для выбора исправления для слова с опечаткой генерируются все варианты УДАЛЕНИЯ (выделение моё) <...>" -- таким образом мы будем редактировать слова со ВСТАВКОЙ, например, "мосЮква" вместо "москва". Чтобы исправлять удаления, нужно было бы в рамках алгоритма ПОДСТАВЛЯТЬ во все возможные позиции все возможеные буквы: чтобы, например, из "мсква" получить "мОсква"

In [16]:
get_corrected_word('солонце', correct_vocab, deletions_vocab)

['солнце', 'солнце', 'солонцы']


'солнце'

In [17]:
get_corrected_word('сонце', correct_vocab, deletions_vocab)

['донце', 'синице', 'соней']


'соней'

Оценка качества

In [18]:
bad = open('sents_with_mistakes.txt', encoding='utf-8').read().splitlines()
true = open('correct_sents.txt', encoding='utf-8').read().splitlines()

# напишем функцию, которая будет сопоставлять слова в правильном и ошибочном варианте
# разобьем предложение по пробелам и удалим пунктуация на границах слов
def align_words(sent_1, sent_2):
    tokens_1 = sent_1.lower().split()
    tokens_2 = sent_2.lower().split()
    
    tokens_1 = [token.strip(punctuation) for token in tokens_1]
    tokens_2 = [token.strip(punctuation) for token in tokens_2]
    
    tokens_1 = [token for token in tokens_1 if token]
    tokens_2 = [token for token in tokens_2 if token]
    
    assert len(tokens_1) == len(tokens_2)
    
    return list(zip(tokens_1, tokens_2))

In [19]:
correct = 0
total = 0

total_mistaken = 0
mistaken_fixed = 0

total_correct = 0
correct_broken = 0

cashed = {}
for i in tqdm(range(len(true))):
    word_pairs = align_words(true[i], bad[i])
    for pair in word_pairs:
        # чтобы два раза не исправлять одно и тоже слово - закешируем его
        # перед тем как считать исправление проверим нет ли его в кеше
        
        predicted = cashed.get(pair[1], get_corrected_word(pair[1], correct_vocab, deletions_vocab))
        cashed[pair[1]] = predicted
        
        if predicted == pair[0]:
            correct += 1
        total += 1
        
        if pair[0] == pair[1]:
            total_correct += 1
            if pair[0] !=  predicted:
                correct_broken += 1
        else:
            total_mistaken += 1
            if pair[0] == predicted:
                mistaken_fixed += 1

[]
['шпионском']
['гламурные']
[]
['slap', 'cape', 'clip', 'cola']
[]
[]
['эпоним']
[]
['язычка']
['олень', 'олень', 'олень', 'олень']
[]
['серую', 'серую']
['отсутствие', 'отсутствие']
['сеневая', 'фоновая', 'попсовая', 'сеневая', 'основах']
[]
[]
['карасо', 'набрано', 'карасо']
['общему']
['щекам', 'яцека', 'ящика']
[]
['предлагаю']
['сегодняшнее']
['дороше', 'хэрше', 'хэрше', 'хороле', 'хороши']
['выходных']
['лите', 'рате', 'ринне', 'рита']
['снялся']
['потому', 'панаму', 'потому', 'патами']
['пытаясь']
['чаше', 'алше']
['дуброве', 'дубравы']
[]
[]
['билетных']
[]
['торто', 'чомом', 'чтят', 'чомом', 'чтят']
[]
['модный']
[]
['дороше', 'хэрше', 'хэрше', 'хороле', 'хороши']
[]
['кормилица']
['участвовать', 'участвовать', 'участвовать']
['сенья', 'седая', 'средн']
[]
[]
['ответственность', 'ответственность', 'ответственность']
[]
['казнена']
['740']
['начального', 'начальные']
['залог', 'зажги']
[]
[]
[]
['легии', 'сего', 'силиг', 'осле']
['цветянка']
[]
['женщина']
[]
[]
['малолетним

['торто', 'комом', 'китти', 'комом', 'китти']
['естественно', 'естественно', 'естественно']
['сенья', 'седая', 'средн']
['принципы', 'принципы', 'принципы']
['применял']
['срываясь']
['естественно', 'естественно', 'естественно']
['одиночество']
['блеклых']
['желанием']
['просмотре', 'промоутер', 'просмотра', 'просмотре']
['конакри', 'конакри', 'контракт']
[]
['естественно', 'естественно', 'естественно']
['нукдан', 'нужда', 'нуква']
[]
['ответьте', 'ответьте', 'ответьте', 'ответьте']
['арестуют', 'андресу']
['барракуд', 'барракуд']
[]
['созванивается']
[]
['естественно', 'естественно', 'естественно']
[]
['пробами', 'пробках']
['минута', 'минутами', 'минута']
[]
[]
['майлз']
[]
['следующая']
[]
[]
['нефте', 'ищете', 'хинт', 'инеем', 'хинт']
['кичо', 'начо', 'нидо']
[]
[]
[]
['шоколаду']
['придается']
['совпали', 'сорвали']
[]
['связывает']
[]
[]
[]
['гомпы', 'копты', 'кормы', 'компу']
['занятных']
[]
['матроны', 'матроны']
[]
[]
['очищенный']
[]
['хрям', 'парам', 'паря']
[]
['волнующей']

In [20]:
print(correct/total)
print(mistaken_fixed/total_mistaken)
print(correct_broken/total_correct)

0.8540270135067534
0.15838509316770186
0.04306879522223498


Результаты слабые, что ожидаемо при использованной упрощённой процедуре. Ниже пробую чуть усложнить процедуру

In [21]:
# если нужно не только удаления, но и вставки: функция из семинара

def get_edits(word):
    
    "Создаем кандидатов, которые отличаются на одну букву"
    
    letters    = 'йцукенгшщзхъфывапролджэячсмитьбюё'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    
    deletes    = [L + R[1:]               for L, R in splits if R]
#     transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
#     replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    
    return set(deletes + inserts)

In [22]:
edits_vocab = dict()
for w in tqdm(correct_vocab):
    for edit in get_edits(w):
        if edit not in edits_vocab:
            edits_vocab[edit] = w

In [23]:
def NEW_get_corrected_word(word, correct_vocab, edits_vocab):
    if predict_mistaken(word, correct_vocab) == False:
        return word
    else:
        candidates = []
        for i in get_edits(word):
            if edits_vocab.get(i):
                candidates.append(edits_vocab.get(i))
        print(candidates)
        
        if len(candidates) == 0:
            return word
        elif len(candidates) == 1:
            return edits_vocab.get(candidates[0])
        else:
            return get_most_probable_word(candidates)

In [27]:
NEW_get_corrected_word('сонце', correct_vocab, edits_vocab)

['сьоне', 'согне', 'сне', 'конце', 'монце', 'сионе', 'солнцем', 'слоне', 'донце', 'соц', 'конце', 'конце', 'сотне', 'соц', 'солнцев', 'сосне', 'сотне', 'сонет', 'монце', 'сонче', 'сонче', 'солнцев', 'донце', 'соней']


'конце'

Хотя "солнце" и не выбирается, по крайней мере оно теперь есть среди кандидатов

In [26]:
NEW_get_corrected_word('масква', correct_vocab, edits_vocab)

'масква'

Такая замена не исправляется в соответствии с процедурой

In [28]:
NEW_get_corrected_word('солмце', correct_vocab, edits_vocab)

['соломе', 'солнце', 'солнце', 'соломе', 'солнце']


'солнце'

А такая -- да 

In [24]:
correct = 0
total = 0

total_mistaken = 0
mistaken_fixed = 0

total_correct = 0
correct_broken = 0

cashed = {}
for i in tqdm(range(len(true))):
    word_pairs = align_words(true[i], bad[i])
    for pair in word_pairs:
        # чтобы два раза не исправлять одно и тоже слово - закешируем его
        # перед тем как считать исправление проверим нет ли его в кеше
        
        predicted = cashed.get(pair[1], NEW_get_corrected_word(pair[1], correct_vocab, edits_vocab))
        cashed[pair[1]] = predicted
        
        if predicted == pair[0]:
            correct += 1
        total += 1
        
        if pair[0] == pair[1]:
            total_correct += 1
            if pair[0] !=  predicted:
                correct_broken += 1
        else:
            total_mistaken += 1
            if pair[0] == predicted:
                mistaken_fixed += 1

[]
['шпионском', 'шпионской', 'шпионской', 'шпионской', 'шпионском']
['гламурные', 'гламурные', 'гламурным', 'гламурные', 'гламурным']
[]
['olap', 'clup', 'clan', 'camp']
['апофеозом', 'апофеозом', 'апофеозом']
[]
['эпоним', 'пони', 'пони']
['получается', 'получаются', 'получаться', 'получаться']
['язычка', 'язычке', 'язык', 'язычок', 'язык', 'язычники', 'язычники', 'язычка', 'язычке']
[]
[]
['сеют', 'сеют', 'сетует', 'советуют', 'сету', 'сетует', 'советуют', 'сету']
['отсутствием', 'отсутствием']
['новая', 'основах', 'носовая', 'основав', 'новая', 'тоновая', 'основания', 'фоновая', 'основ', 'основах', 'основав', 'основал', 'носовая', 'основал', 'основал', 'основам', 'основан', 'основан', 'основам']
[]
[]
['набрано', 'напрасной', 'напрасной', 'набрано', 'напрасное']
['вообще', 'общему', 'всеобщем', 'всеобщем', 'общему', 'обще']
['щекам', 'ящика', 'яцека', 'ящика', 'щенка', 'щелка', 'ящика', 'яцека', 'ека', 'щеках', 'ека']
[]
['предлагают', 'предаю', 'предлагают', 'предаю']
[]
['хорошем

['наверх', 'сверху', 'вверху', 'сверху', 'вверху', 'наверх', 'веру', 'кверху', 'веху', 'веху', 'кверху']
['поддерживаемая', 'поддерживаемая', 'поддерживаемых', 'поддерживаемым', 'поддерживающем', 'поддерживаемое', 'поддерживаемую', 'поддерживаемую', 'поддерживаемым', 'поддерживаемой', 'поддерживает', 'поддерживаемой', 'поддерживает', 'поддерживаемым', 'поддерживающем', 'поддерживает']
['жалеть', 'жалеть']
['проявить', 'проявить', 'правит', 'проявить', 'справить', 'вправить', 'справить']
[]
['успокоить', 'успокоить', 'успокоить']
[]
['беседня', 'беседня', 'сведя', 'ясеня', 'следя', 'средняя', 'сведя', 'седая', 'сегодня', 'средн', 'дня', 'сегодня', 'дня', 'сенья', 'седан', 'сенья', 'седан']
[]
['милтон', 'льон', 'миньон', 'мило', 'мион', 'мильян', 'семильон', 'мильян', 'мильтона', 'миньон', 'семильон', 'мильтона', 'льон', 'мильтону', 'миль', 'мило']
['ване', 'вале', 'варище', 'варище', 'чаще', 'вате', 'ваше', 'ве', 'вазе', 'ваке', 'чаще', 'ваге', 'вале', 'варе', 'вате', 'ва', 'чаще', 'ва

In [25]:
print(correct/total)
print(mistaken_fixed/total_mistaken)
print(correct_broken/total_correct)

0.8419209604802401
0.18711180124223603
0.06121511427586999


Прирост качества получился незначительным, зато я постарался исправить кажущуюся некорректность